In [20]:
# The code was removed by DSX for sharing.

0     After waiting what seemed like  G3   forever ...
1     Yeah   G10   it was such a change I  G9   swo...
2       G8  And I was so glad that all of the measu...
3     Ben and I had been saying for the past two we...
4     Of course   G13  that is what happened for th...
Name: sentence, dtype: object

In [21]:
_PADDING_VALUE = 4

In [22]:
def process_sentences(sentence):
    
    #make all words lowercase
    sentence_lower = sentence.lower()
    
    #string to list of words
    sentence_separated = sentence_lower.split()
    
    #remove non alphabetic words
    sentence_alphabetic = [word for word in sentence_separated if word.isalpha() or re.search(r'g\d{1,2}',word) is not None]

    #pad the sentence with four 0s
    sentence_padded = ['0']*_PADDING_VALUE+sentence_alphabetic
    
    processed_sentence = np.array(sentence_padded)
    return sentence_alphabetic    

In [23]:
#df to list
sentences_list = df_data.values.tolist()

processed_sentences = [process_sentences(sent[0]) for sent in sentences_list]

In [24]:
length = _PADDING_VALUE+1
sequences = list()
for sentence in processed_sentences:
    for i in range(length, len(sentence)):
        #select the sequence of words
        seq = sentence[i-length:i]
        # convert into a line
        line = ' '.join(str(v) for v in seq)
        #store
        sequences.append(line)
print('Total Sequences: %d' % len(sequences))

Total Sequences: 18667


In [25]:
sequences[:10]

['after waiting what seemed like',
 'waiting what seemed like g3',
 'what seemed like g3 forever',
 'seemed like g3 forever to',
 'like g3 forever to get',
 'g3 forever to get into',
 'forever to get into the',
 'to get into the room',
 'get into the room we',
 'into the room we g19']

In [26]:
from numpy import array
from pickle import dump
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding

Using TensorFlow backend.


In [27]:
#integer encode sequences of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sequences)
numerical_sequences = tokenizer.texts_to_sequences(sequences)

In [28]:
# vocabulary size
vocab_size = len(tokenizer.word_index) + 1
vocab_size

2982

In [30]:
# separate into input and output
numerical_sequences = array(numerical_sequences)
X, y = numerical_sequences[:,:-1], numerical_sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]

In [34]:
# define model
model = Sequential()
model.add(Embedding(vocab_size, 4, input_length=seq_length))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 4, 4)              11928     
_________________________________________________________________
lstm_1 (LSTM)                (None, 4, 100)            42000     
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 2982)              301182    
Total params: 445,610
Trainable params: 445,610
Non-trainable params: 0
_________________________________________________________________
None


In [35]:
# compile model
create_model = model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
fit_model = model.fit(X, y, batch_size=128, epochs=100)

Epoch 1/100
18667/18667 [==============================] - 11s - loss: 6.8491 - acc: 0.0448    
Epoch 2/100
18667/18667 [==============================] - 11s - loss: 6.3680 - acc: 0.0494    
Epoch 3/100
18667/18667 [==============================] - 10s - loss: 6.3314 - acc: 0.0494    
Epoch 4/100
18667/18667 [==============================] - 10s - loss: 6.3060 - acc: 0.0494    
Epoch 5/100
18667/18667 [==============================] - 10s - loss: 6.2904 - acc: 0.0494    
Epoch 6/100
18667/18667 [==============================] - 10s - loss: 6.2793 - acc: 0.0481    
Epoch 7/100
18667/18667 [==============================] - 10s - loss: 6.2579 - acc: 0.0496    
Epoch 8/100
18667/18667 [==============================] - 10s - loss: 6.2410 - acc: 0.0496    
Epoch 9/100
18667/18667 [==============================] - 10s - loss: 6.2292 - acc: 0.0494    
Epoch 10/100
18667/18667 [==============================] - 10s - loss: 6.2115 - acc: 0.0498    
Epoch 11/100
18667/18667 [=============

In [115]:
whole_sentence = 'the person you want to meet will be downstairs in a second .'
#select a seed text
#seed_text = lines[randint(0,len(lines))]
seed_text = 'the person you want'
print(seed_text + '\n')

the person you want



In [119]:
# generate a sequence from a language model
def generate_seq(model, tokenizer, seq_length, seed_text, whole_sentence, n_words):
    result = [seed_text]
    count = seq_length
    in_text = seed_text
    whole_sentence = whole_sentence.split()
    # generate a fixed number of words
    for _ in range(seq_length, len(whole_sentence)):
        # encode the text as integer
        encoded = tokenizer.texts_to_sequences([in_text])[0]
        # truncate sequences to a fixed length
        encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
        # predict probabilities for each word
        yhat = model.predict_classes(encoded, verbose=0)
        # map predicted word index to word
        out_word = ''
        for word, index in tokenizer.word_index.items():
            if index == yhat:
                out_word = word
                break
        print(out_word)
        if re.search(r'g\d{1,2}',out_word) is not None:
            # append to input
            in_text += ' ' + out_word
            result.append(out_word)
        else:
            next_word = whole_sentence[count]
            in_text += ' ' + next_word
            result.append(next_word)
            count += 1
    return ' '.join(result)

In [120]:
resulting_sentence = generate_seq(model, tokenizer, seq_length, seed_text, whole_sentence, 10)

perhaps
get
you
g14
a
nothing
nothing
through
peas


In [1]:
resulting_sentence

NameError: name 'resulting_sentence' is not defined

In [132]:
!ls '../LSTM'

model.h5


In [131]:
# save the model to file
model.save('../LSTM/model.h5')

In [18]:
service_path = "https://ibm-watson-ml.eu-gb.bluemix.net"
username = "cf3630f1-60f6-424e-9e28-069fbc198b86"
password = "f6db3b3e-422f-46f8-a702-054b095e489a"
instance_id = "76b51224-c4a5-4ebf-8bd3-ad26309ea5a5"

In [21]:
import urllib3, requests, json

headers = urllib3.util.make_headers(basic_auth='{}:{}'.format(username, password))
url = '{}/v3/identity/token'.format(service_path)
response = requests.get(url, headers=headers)
mltoken = 'Bearer ' + json.loads(response.text).get('token')

In [24]:
endpoint_instance = service_path + "/v3/wml_instances/" + instance_id
header = {'Content-Type': 'application/json', 'Authorization': mltoken}

response_get_instance = requests.get(endpoint_instance, headers=header)
print(response_get_instance)
print(response_get_instance.text)

<Response [200]>
{"metadata":{"guid":"76b51224-c4a5-4ebf-8bd3-ad26309ea5a5","url":"https://instances/v3/wml_instances/76b51224-c4a5-4ebf-8bd3-ad26309ea5a5","created_at":"2018-05-14T09:57:26.088Z","modified_at":"2018-05-14T09:57:26.088Z"},"entity":{"source":"Bluemix","published_models":{"url":"https://ibm-watson-ml.eu-gb.bluemix.net/v3/wml_instances/76b51224-c4a5-4ebf-8bd3-ad26309ea5a5/published_models"},"usage":{"expiration_date":"2018-06-01T00:00:00.000Z","computation_time":{"limit":180000,"current":0},"model_count":{"limit":200,"current":0},"prediction_count":{"limit":5000,"current":0},"gpu_count":{"limit":8,"current":0},"capacity_units":{"limit":180000000,"current":0},"deployment_count":{"limit":5,"current":0}},"plan_id":"3f6acf43-ede8-413a-ac69-f8af3bb0cbfe","status":"Active","organization_guid":"812f63ef-afd1-4d70-b6ee-c549b869eb09","region":"eu-gb","account":{"id":"c8e459ba42cd1088b0badad838a0b2e5","name":"vu amsetrdam","type":"TRIAL"},"owner":{"ibm_id":"50W8DA7BU3","email":"m.tu

In [26]:
endpoint_published_models = json.loads(response_get_instance.text).get('entity').get('published_models').get('url')
print(endpoint_published_models)

https://ibm-watson-ml.eu-gb.bluemix.net/v3/wml_instances/76b51224-c4a5-4ebf-8bd3-ad26309ea5a5/published_models


In [27]:
header = {'Content-Type': 'application/json', 'Authorization': mltoken}

response_get = requests.get(endpoint_published_models, headers=header)
print(response_get)
print(response_get.text)

<Response [200]>
{"limit":1000,"resources":[],"first":{"url":"https://deployment/v3/wml_instances/76b51224-c4a5-4ebf-8bd3-ad26309ea5a5/published_models?limit=1000"}}


In [28]:
[endpoint_deployments] = [x.get('entity').get('deployments').get('url') 
for x in json.loads(response_get.text).get('resources') if x.get('metadata').get('guid') == saved_model.uid]

print(endpoint_deployments)

ValueError: not enough values to unpack (expected 1, got 0)